Привет, меня зовут Люман Аблаев. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ..
</div>
<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>


<font color='orange' style='font-size:24px; font-weight:bold'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
- Приятное оформление выводов и структурированность кода в целом!  Приятно проверять такие работы.
- Тебе удалось справиться с текущей задачей и достигнуть неплохих результатов, поздравляю!
* Отправляю проект назад, если у тебя появились вопросы. Если их нет, то можешь просто отправить проект еще раз и я его зачту.




# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

# Импорт необходимых библиотек

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, isnan, count
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline






<div class="alert alert-block alert-success">
<b>Успех:</b> Импорты на месте
</div>


## Инициализация локальной Spark-сессии


In [2]:
spark = SparkSession.builder \
                    .master("local") \
                    .appName("housing_regression") \
                    .getOrCreate()

## Чтение содержимого файла


In [3]:
df = spark.read.csv("/datasets/housing.csv", inferSchema=True, header=True)


<div class="alert alert-block alert-success">
<b>Успех:</b> Данные успешно загружены.
</div>


## Вывод типов данных колонок датасета


In [4]:
df.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)



In [5]:
# Вывод количества строк в DataFrame
print("Количество строк в DataFrame: ", df.count())


Количество строк в DataFrame:  20640


## Предобработка данных
### Исследование данных на наличие пропусков и их заполнение

In [6]:
# Проверка колонок датасета на пропуски и вывод только столбцов с пропусками
missing_data = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns])
missing_data = missing_data.select([c for c in missing_data.columns if missing_data.select(c).first()[c] > 0])
missing_data.show()


+--------------+
|total_bedrooms|
+--------------+
|           207|
+--------------+



In [7]:
# Удаление строк с пропусками
df = df.na.drop()

# Проверка количества строк после удаления пропусков
print("Количество строк в DataFrame после удаления пропусков: ", df.count())


Количество строк в DataFrame после удаления пропусков:  20433


<div class="alert alert-block alert-success">
<b>Успех:</b> Идея с удалением - допустима, но если есть возможно - лучше обрабатывать
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Не стал заполнять ибо сначала подумал что речь о квартире, а там зависит от многих других факторов такие как площать и количества комнат))) хотя даже если о жилом комплексе сильно может вальироваться(мб студий много и всё такое) а вы нас пока не научили делать в спарке тоже исследование, так что так и так проще удалить раз их всего 207
</div>


<div class="alert alert-block alert-success">
<b>Успех[2]: </b> Принял. Для заполнения пропусков можно использовать <a href="https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Imputer.html">Imputer</a>
</div>


In [8]:
df.describe().toPandas() 

,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20433,20433,20433,20433,20433,20433,20433,20433,20433,20433
1,mean,-119.57068859198068,35.63322125972706,28.633093525179856,2636.5042333480155,537.8705525375618,1424.9469485635982,499.43346547251997,3.8711616013312273,206864.41315519012,None
2,stddev,2.003577890751096,2.1363476663779872,12.591805202182835,2185.269566977601,421.38507007403115,1133.2084897449597,382.2992258828481,1.899291249306247,115435.66709858322,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично! Выведены типы данных и информация о колонках и toPAndas использован к аггрегированной таблице.
</div>

### Преобразование

#### Создаем индексатор и кодировщик для категориальных колонок

In [29]:
# Преобразование категориальной колонки с помощью One Hot Encoding
indexer = StringIndexer(inputCol="ocean_proximity", outputCol="ocean_proximity_index")
encoder = OneHotEncoder(inputCols=["ocean_proximity_index"], outputCols=["ocean_proximity_vec"])

#### Создаем StandardScaler для каждой числовой колонки

In [30]:
# Масштабирование числовых признаков
assembler = VectorAssembler(inputCols=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'ocean_proximity_vec'], outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

### Разделение данных на обучающую и тестовую выборки

In [31]:
train_data, test_data = df.randomSplit([.8,.2], seed=13)

## Построение моделей линейной регрессии
### Модель с использованием всех данных из файла


In [32]:
# Построение модели линейной регрессии
lr = LinearRegression(featuresCol='scaledFeatures', labelCol='median_house_value')

# Пайплайн для обработки данных и обучения модели
pipeline = Pipeline(stages=[indexer, encoder, assembler, scaler, lr])

# Обучение модели на всех данных
model = pipeline.fit(train_data)


24/04/18 07:02:17 WARN Instrumentation: [f23cd1dd] regParam is zero, which might cause numerical instability and overfitting.


<div class="alert alert-block alert-success">
<b>Успех:</b> Лайк за пайплайн - все препроцессоры будут успешно обучены без утечек информации.
</div>

### Модель с использованием только числовых переменных


In [33]:
# Построение модели только на числовых переменных
assembler_num = VectorAssembler(inputCols=['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income'], outputCol="features")
pipeline_num = Pipeline(stages=[assembler_num, scaler, lr])

# Обучение модели только на числовых переменных
model_num = pipeline_num.fit(train_data)

24/04/18 07:02:19 WARN Instrumentation: [f2d64d79] regParam is zero, which might cause numerical instability and overfitting.



<div class="alert alert-block alert-success">
<b>Успех: </b> Обучены две модели в соответствии с заданием.
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> 
    сильно смущает, что с этим обычно делают?
    
    24/04/17 15:46:33 WARN Instrumentation: [fa7f1a5b] regParam is zero, which might cause numerical instability and overfitting.
    
    24/04/17 15:46:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
    
    24/04/17 15:46:33 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
    
    24/04/17 15:46:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
    
    24/04/17 15:46:34 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
</div>


<div class="alert alert-block alert-success">
<b>Ответ[2]: </b> Устанавливают regParam=0.01 например для линейной регресии. А вот следующие логи я запредпродюсить не смог - связано как-будто с библиотеками.
</div>


## Сравнение результатов работы линейной регрессии


In [34]:
predictions = model.transform(test_data)

# Оценка качества модели
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
evaluator = RegressionEvaluator(labelCol="median_house_value", predictionCol="prediction", metricName="r2")
r2 = evaluator.evaluate(predictions)

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")
print(f"R2: {r2}")

RMSE: 68209.9896597888
MAE: 49734.5085422155
R2: 0.6503048673505825


In [35]:
predictions_num = model_num.transform(test_data)

# Оценка качества модели только на числовых переменных
rmse_num = evaluator.evaluate(predictions_num, {evaluator.metricName: "rmse"})
mae_num = evaluator.evaluate(predictions_num, {evaluator.metricName: "mae"})
r2_num = evaluator.evaluate(predictions_num, {evaluator.metricName: "r2"})

print(f"RMSE (только числовые переменные): {rmse_num}")
print(f"MAE (только числовые переменные): {mae_num}")
print(f"R2 (только числовые переменные): {r2_num}")



RMSE (только числовые переменные): 69020.29866042463
MAE (только числовые переменные): 50662.23830843402
R2 (только числовые переменные): 0.6419470233876452


In [36]:
# Сравнение результатов моделей
if rmse < rmse_num:
    print("Модель со всеми данными показывает лучший результат по метрике RMSE.")
else:
    print("Модель только с числовыми переменными показывает лучший результат по метрике RMSE.")

if mae < mae_num:
    print("Модель со всеми данными показывает лучший результат по метрике MAE.")
else:
    print("Модель только с числовыми переменными показывает лучший результат по метрике MAE.")

if r2 > r2_num:
    print("Модель со всеми данными показывает лучший результат по метрике R2.")
else:
    print("Модель только с числовыми переменными показывает лучший результат по метрике R2.")


Модель со всеми данными показывает лучший результат по метрике RMSE.
Модель со всеми данными показывает лучший результат по метрике MAE.
Модель со всеми данными показывает лучший результат по метрике R2.


In [37]:
# Сравнение результатов
print(f"Разница RMSE: {rmse - rmse_num}")
print(f"Разница MAE: {mae - mae_num}")
print(f"Разница R2: {r2 - r2_num}")

# Завершение Spark сессии
spark.stop()

Разница RMSE: -810.3090006358252
Разница MAE: -927.7297662185156
Разница R2: 0.008357843962937261


**Общий вывод о проделанной работе и результатах:**

В ходе проекта была выполнена работа по обучению модели линейной регрессии для предсказания медианной стоимости жилья в Калифорнии на основе данных 1990 года. Были выполнены следующие этапы:

1. **Инициализация Spark-сессии** и чтение данных из файла.
2. **Предобработка данных**, включая обработку пропусков и преобразование категориальных переменных с помощью One Hot Encoding.
3. **Масштабирование числовых признаков** и разделение данных на обучающую и тестовую выборки.
4. **Построение двух моделей линейной регрессии**: одна с использованием всех данных, вторая — только на числовых переменных.
5. **Оценка качества моделей** с помощью метрик RMSE, MAE и R2.

**Результаты:**
- Модель, обученная на всех данных, показала лучшие результаты по всем метрикам по сравнению с моделью, обученной только на числовых переменных:
  - **RMSE**: 68209.99 против 69020.30
  - **MAE**: 49734.51 против 50662.24
  - **R2**: 0.6503 против 0.6419

Это говорит о том, что включение категориальных переменных после их преобразования в числовые улучшает точность модели. В целом, модель справилась с задачей предсказания стоимости жилья, хотя есть потенциал для дальнейшего улучшения показателей, возможно, за счет более сложных методов обработки данных или использования других алгоритмов машинного обучения.

После завершения работы с моделью Spark-сессия была корректно закрыта.

<div class="alert alert-block alert-success">
<b>Успех: </b> Все метрики корректно вычислены и проаналазированы, молодец!
</div>


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> П.С. Проект можно зачесть я всёравно даже принятые захожу читаю коменты ревьюера, вы обычно полузную информацию всегда пишите даже если замечаний нет😁👍
</div>

<div class="alert alert-block alert-success">
    
    
<b>Успех[2]:</b> 🤝 Вот тебе немного полезного материала по теме:

    
- https://sparkbyexamples.com/pyspark-tutorial/
- https://sparkbyexamples.com/
- https://github.com/dvgodoy/handyspark
- https://www.tutorialspoint.com/pyspark/index.htm
- https://www.guru99.com/pyspark-tutorial.html
- https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data
- https://www.geeksforgeeks.org/hadoop-tutorial/
</div>
